In [1]:
%matplotlib inline
import importlib
import pytorch_utils_oh_1; importlib.reload(pytorch_utils_oh_1); from pytorch_utils_oh_1 import *;

Pytorch utils oh: pytorch_utils_oh_1.py
Pytorch: 0.2.0_4
Loadig pytorch_utils_oh defaults
Pytorch utils oh: pytorch_utils_oh_1.py
Pytorch: 0.2.0_4
Loadig pytorch_utils_oh defaults


# Data loading

In [2]:
all_data_org = pd.read_csv('data/en_train_org.csv')

In [3]:
all_data_org[pd.isnull(all_data_org['before'])][:3]
all_data = all_data_org.dropna()
print("Data rows: {},  (dropped none rows: {})".format(len(all_data), len(all_data_org)-len(all_data)))

Data rows: 9918390,  (dropped none rows: 51)


In [4]:
all_data = all_data[all_data['class'] != 'VERBATIM']
print("Data rows: {},  (dropped (verbatim) rows: {})".format(len(all_data), len(all_data_org)-len(all_data)))
all_data = all_data.reset_index(drop=True)
# Note we dropped VERBATIM class. Thats because it had so many weird characters.

Data rows: 9840282,  (dropped (verbatim) rows: 78159)


In [5]:
all_data_sentence_index = all_data.set_index('sentence_id')

In [6]:
number_classes = ['DATE','CARDINAL','MEASURE','ORDINAL','DECIMAL','MONEY', 'DIGIT', 'TELEPHONE', 'TIME', 'FRACTION', 'ADDRESS']

In [7]:
number_data = all_data[all_data['class'].isin(number_classes)]
print("Data rows: {},  (dropped rows: {})".format(len(number_data), len(all_data_org)-len(number_data)))
number_data = number_data.reset_index(drop=True)

Data rows: 448176,  (dropped rows: 9470265)


### More balanced sample

In [8]:
balanced_data_classes_select = list(number_data.groupby('class'))
def balanced_data_randomize(max_len=10000):
    global balanced_data
    balanced_data = pd.concat([v.sample(min(max_len, len(v))) for k, v in balanced_data_classes_select])
balanced_data_randomize()

### Letters all

In [9]:
tmp = sorted(list(set(''.join(number_data['before']))))
letters_all = ['<EOS>', '<SOS>'] + sorted(list(set(tmp)))
letters_all_index = dict((c, i) for i, c in enumerate(letters_all))
print(''.join(letters_all))
print(len(letters_all))

<EOS><SOS> "$%'(),-./0123456789:ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz£¥ª²³µº¼½¾Ωμ€⅓⅔⅛⅝⅞
94


### Number words

In [10]:
arr = list(set(list(number_data['after'])))
arr = [s.split(' ') for s in arr]
arr = np.concatenate(arr)
arr = list(set(arr))
number_words = ['<EOS>', '<SOS>'] + arr
number_words_index = dict((c, i) for i, c in enumerate(number_words))
len(number_words)

546

In [11]:
def number_words_to_tensor(words, include_eos=True):
    return words_to_tensor(words, words_lookup_index=number_words_index, include_eos=include_eos)
number_words_to_tensor(['one', 'first']).shape

(1, 3, 546)

In [12]:
number_words_onehot_sos = number_words_to_tensor([SOS_TOKEN], include_eos=False)
number_words_onehot_sos = Variable(torch.from_numpy(number_words_onehot_sos)).cuda()
number_words_onehot_sos.size()

torch.Size([1, 1, 546])

### Sample

In [13]:
def get_random_sample():
    sample_row = balanced_data.iloc[random.randint(1, len(balanced_data)-1)]
    sentence_id = sample_row['class']

    rows = all_data_sentence_index.loc[sample_row['sentence_id']]
    befores = rows.before.values
        
    token_id_idx = list(rows['token_id']).index(sample_row['token_id'])
    befores[token_id_idx] = SAMPLE_WORD_TOKEN

    return sample_row['before'], sample_row['after'], sample_row['class'], befores
            
def tmp():
    s_bef, s_aft, s_class, s_sentence = get_random_sample()
    print(s_class, ':', s_bef, '->', s_aft)
    print(' '.join(s_sentence))
    print(sentence_word_vectorize(s_sentence).shape)
    print(number_words_to_tensor(s_aft.split(' ')).shape)
tmp()

ORDINAL : 8th -> eighth
Pierre Gallo Sex : Male Age : 18 Position : The 6th section of the <SAMPLE> experiment division's aviation unit .
(1, 21, 50)
(1, 2, 546)


In [ ]:
sentence_word_vectorize_no_defaults

In [14]:
%%timeit
get_random_sample()

419 µs ± 5.16 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# Model functions

In [15]:
use_cuda = True

### Encoder

In [22]:
class EncoderRNN(nn.Module):
    def __init__(self, wordvect_size, chars_input_size, words_hidden_size, chars_hidden_size,
                 words_layers=1, chars_layers=1):
        super(EncoderRNN, self).__init__()
        
        self.words_layers = words_layers
        self.chars_layers = chars_layers
        self.words_hidden_size = words_hidden_size
        self.chars_hidden_size = chars_hidden_size

        #self.rnn_words = nn.LSTM(wordvect_size, words_hidden_size // 2, words_layers,
        #                         batch_first=True, bidirectional=True)

        self.rnn_chars = nn.LSTM(chars_input_size, chars_hidden_size // 2, chars_layers,
                                batch_first=True, bidirectional=True)
        
    def forward(self, word_vectors, string_tensor, hidden = None, init_hidden = True):
        if init_hidden:
            hidden_words, hidden_chars = self.init_hidden()
        
        #all_outputs_words, hidden_words = self.rnn_words(word_vectors, hidden_words)
        #output_words = all_outputs_words[:, -1]
        
        all_outputs_chars, hidden_chars = self.rnn_chars(string_tensor, hidden_chars)
        output_chars = all_outputs_chars[:, -1]
        
        # output = torch.cat((output_words, output_chars), 1)
        # output = torch.cat((output_chars), 1)
        output = output_chars
        
        return output

    def init_hidden(self):
        var1_1 = Variable(torch.zeros(2 * self.words_layers, 1, self.words_hidden_size // 2))
        var1_2 = Variable(torch.zeros(2 * self.words_layers, 1, self.words_hidden_size // 2))
        var2_1 = Variable(torch.zeros(2 * self.chars_layers, 1, self.chars_hidden_size // 2))
        var2_2 = Variable(torch.zeros(2 * self.chars_layers, 1, self.chars_hidden_size // 2))
        if use_cuda:
            var1_1 = var1_1.cuda(); var1_2 = var1_2.cuda()
            var2_1 = var2_1.cuda(); var2_2 = var2_2.cuda()
        return ((var1_1, var1_2), (var2_1, var2_2))

In [23]:
def get_encoder(debug=False):
    # s_bef, s_aft, s_class, s_word_vs = get_random_sample()
    s_bef, s_aft, s_class, s_sentence = get_random_sample()
    
    s_word_vs = sentence_word_vectorize(s_sentence)
    s_string = string_to_tensor(s_bef, letters_all_index)
    target = number_words_to_tensor(s_aft.split(' '))
    
    encoder_rnn = EncoderRNN(wordvect_size=s_word_vs.shape[-1], chars_input_size=len(letters_all),
                                      words_hidden_size=128, chars_hidden_size=128,
                                      words_layers=2, chars_layers=2).cuda()
    
    output_encoded = encoder_rnn(Variable(torch.from_numpy(s_word_vs)).cuda(), Variable(s_string).cuda())
    if debug:
        print('Word vect size:', s_word_vs.shape, '. String vector size:', s_string.size())
        print('Output:', output_encoded.size())
    return encoder_rnn, output_encoded;
encoder_rnn, output_encoded = get_encoder(debug=True)

Word vect size: (1, 21, 50) . String vector size: torch.Size([1, 5, 94])
Output: torch.Size([1, 128])


### Decoder

In [24]:
class DecoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1):
        super(DecoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        
        self.rnn = nn.GRU(input_size, hidden_size, n_layers,
                                 batch_first=True, bidirectional=False)
                         # LSTM would require own hidden included
        
        self.lin_out = nn.Linear(hidden_size, input_size)
        #self.softmax = nn.LogSoftmax()

    def forward(self, char, hidden):
        #char = char.view(1,1,-1)
        #hidden = hidden.view(1,1,-1)
        output, hidden = self.rnn(char, hidden)
        output = output[:, -1] # view(1,-1)
        output = self.lin_out(output)
        output = F.log_softmax(output)
        return output, hidden

decoder_rnn = DecoderRNN(input_size=len(number_words), hidden_size=output_encoded.size()[-1], n_layers=1)
decoder_rnn = decoder_rnn.cuda()
decoder_rnn

DecoderRNN (
  (rnn): GRU(546, 128, batch_first=True)
  (lin_out): Linear (128 -> 546)
)

In [25]:
tmp_a, tmp_b = decoder_rnn(number_words_onehot_sos, output_encoded.view(1,1,-1))
print(tmp_a.size())
print(tmp_a.topk(1)[0])

torch.Size([1, 546])
Variable containing:
-6.1598
[torch.cuda.FloatTensor of size 1x1 (GPU 0)]



In [26]:
tmp_input = number_words_to_tensor([number_words[tmp_a.topk(1)[1].data[0][0]]])
tmp_input = Variable(torch.from_numpy(tmp_input)).cuda()
tmp = decoder_rnn(tmp_input, tmp_b)
[t.size() for t in tmp]

[torch.Size([1, 546]), torch.Size([1, 1, 128])]

# Training etc

### Accuracy

In [28]:
def evaluate(encoder, decoder, s_bef, s_sentence, max_length=20):
    encoder.eval()
    decoder.eval()
    #s_word_vs = sentence_word_vectorize(s_sentence)
    s_bef_string = string_to_tensor(s_bef, letters_all_index, include_eos=True)

    #encoder_output = encoder(Variable(torch.from_numpy(s_word_vs)).cuda(), Variable(s_bef_string).cuda())
    encoder_output = encoder(None, Variable(s_bef_string).cuda())
    encoder_output = encoder_output.view(1,1,-1)
    
    decoder_hidden = encoder_output
    decoder_input = number_words_onehot_sos

    decoded_output = []
    for _ in range(max_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)

        topv, topi = decoder_output.data.topk(1)
        word_index = topi[0][0]
        word = number_words[word_index] # Use own prediction as next input
                
        if word == '<EOS>':
            break

        decoded_output.append(word)
        
        decoder_input = number_words_to_tensor([word], include_eos=False)
        decoder_input = Variable(torch.from_numpy(decoder_input)).cuda()

    return ' '.join(decoded_output)

evaluate(encoder_rnn, decoder_rnn, '12th', 'he was <SAMPLE>')

'rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs'

In [29]:
def test_model_accuracy(n_sample=10000, print_wrongs=False):
    balanced_data_randomize()
    n_correct = 0
    for iteration in range(n_sample):
        s_bef, s_aft, s_class, s_sentence = get_random_sample()
        output = evaluate(encoder_rnn, decoder_rnn, s_bef, s_sentence)
        if s_aft == output:
            n_correct += 1
        else:
            if print_wrongs:
                print("{:<20} -> {} \n{:<20} != {}".format(s_bef, output, '', s_aft))
                

    print("Accuracy: {:>4.2%} ({:>8d}/{:>8d})".format(
            n_correct/n_sample, n_correct, n_sample))

test_model_accuracy(4, print_wrongs=True)

.78                  -> rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs 
                     != point seven eight
19th                 -> rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs 
                     != nineteenth
25%                  -> rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs 
                     != twenty five percent
102.9                -> rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs rupiahs 
                     != one hundred two point nine
Accuracy: 0.00% (       0/       4)


In [ ]:
%%timeit
test_model_accuracy

### Training

In [31]:
def train(s_bef, s_aft, s_sentence, encoder, decoder, encoder_optimizer, decoder_optimizer, loss_function,
          use_teacher_forcing, max_length=20):

    #s_word_vs = sentence_word_vectorize(s_sentence)
    s_bef_string = string_to_tensor(s_bef, letters_all_index, include_eos=True)
    target_arr = s_aft.split(' ') + ['<EOS>']
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    loss = 0

    #encoder_output = encoder(Variable(torch.from_numpy(s_word_vs)).cuda(), Variable(s_bef_string).cuda())
    encoder_output = encoder(None, Variable(s_bef_string).cuda())
    encoder_output = encoder_output.view(1,1,-1)
    decoder_hidden = encoder_output

    decoder_input = number_words_onehot_sos

    decoded_output = []
    for i in range(len(target_arr)):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)

        decoder_target = number_words_index[target_arr[i]]
        #print(decoder_target)
        decoder_target = Variable(torch.LongTensor([decoder_target])).cuda()
        
        # import IPython; IPython.core.debugger.set_trace()
        
        loss += loss_function(decoder_output, decoder_target)

        topv, topi = decoder_output.data.topk(1)
        word_index = topi[0][0]
        word = number_words[word_index] # Use own prediction as next input
        decoded_output.append(word)
        
        if use_teacher_forcing:
            word = target_arr[i] # replace input with right target
        else:
            # use output normally as input 
            if word == '<EOS>':
                break

        decoder_input = number_words_to_tensor([word], include_eos=False)
        decoder_input = Variable(torch.from_numpy(decoder_input)).cuda()

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return((loss.data[0] / len(target_arr)), ' '.join(decoded_output))
#print(train())

In [32]:
model_train_iterations = 0
model_train_history = []

In [33]:
def train_iterations(n_iters=100000, lr=0.001, teacher_forcing_ratio=0.5,
                     print_every=10000, plot_every=1000):
    global model_train_iterations
    global model_train_history
    start = time.time()
    decoder_rnn.train()
    encoder_rnn.train()

    current_loss = 0
    current_loss_iter = 0

    encoder_optimizer = torch.optim.Adam(encoder_rnn.parameters(), lr=lr)
    decoder_optimizer = torch.optim.Adam(decoder_rnn.parameters(), lr=lr)
    loss_function = nn.NLLLoss()
    
    for iteration in range(1, n_iters + 1):
        model_train_iterations += 1

        use_teacher_forcing = random.random() < teacher_forcing_ratio
        
        s_bef, s_aft, s_class, s_sentence = get_random_sample()
        
        loss, result = train(s_bef=s_bef, s_aft=s_aft, s_sentence=s_sentence,
                             encoder=encoder_rnn, decoder=decoder_rnn,
                             encoder_optimizer=encoder_optimizer, decoder_optimizer=decoder_optimizer,
                             loss_function=nn.NLLLoss(), use_teacher_forcing=use_teacher_forcing,
                             max_length=40 )

        current_loss += loss
        current_loss_iter += 1

        # Print iter number, loss, name and guess
        if iteration % print_every == 0:
            teacher_forcing_str = ""
            if use_teacher_forcing:
                teacher_forcing_str = "(forcing)"
            print("{:>6d} {:>4.0%} ({:>8}) {:>7.3f}   | {:>6.2f}: {} -> {} {} ({})".format(
                      model_train_iterations, iteration/n_iters, time_since(start),
                      current_loss/current_loss_iter, loss,
                      s_bef, teacher_forcing_str, result, s_aft))

        # Add current loss avg to list of losses
        if iteration % plot_every == 0:
            model_train_history.append((current_loss / plot_every, lr))
            current_loss = 0
            current_loss_iter = 0
            
        if model_train_iterations % 10000 == 0:
            balanced_data_randomize()
            
        if model_train_iterations % 50000 == 0:
            test_model_accuracy()
            save_model('numbers_gen_3', encoder_rnn, decoder_rnn, model_train_iterations)
    
    # test_model_accuracy(model, n_sample=10000)

In [34]:
train_iterations(n_iters=10, print_every=5, teacher_forcing_ratio=1)

     5  50% (   0m 0s)   6.291   |   6.23: 13th -> (forcing) rupiahs rupiahs (thirteenth)
    10 100% (   0m 0s)   6.272   |   6.21: 20th -> (forcing) <EOS> <EOS> (twentieth)


In [35]:
train_iterations(n_iters=(10000-model_train_iterations), print_every=1000, teacher_forcing_ratio=0.5)

  1010  10% (   0m 9s)   2.768   |   1.35: 3 -> (forcing) two <EOS> (three)
  2010  20% (  0m 18s)   2.741   |   2.10: 1900 -> (forcing) two <EOS> <EOS> (nineteen hundred)
  3010  30% (  0m 28s)   2.564   |   5.11: 57/100 ->  one hundred thousand five <EOS> (fifty seven one hundredths)
  4010  40% (  0m 38s)   2.400   |   1.49: 2.9% ->  one point one <EOS> (two point nine percent)
  5010  50% (  0m 48s)   2.280   |   1.69: 1.50 ->  two point five <EOS> (one point five o)
  6010  60% (  0m 58s)   2.150   |   2.40: 3.05 mm -> (forcing) one point one five <EOS> <EOS> (three point o five millimeters)
  7010  70% (   1m 9s)   2.064   |   0.78: 125 ->  one hundred <EOS> (one hundred twenty five)
  8010  80% (  1m 19s)   1.886   |   1.83: 70.3 ->  zero point five <EOS> (seventy point three)
  9010  90% (  1m 29s)   1.765   |   0.77: 10 -> (forcing) one <EOS> (ten)


In [36]:
train_iterations(n_iters=50000, teacher_forcing_ratio=0.5)

 20000  20% (  1m 43s)   1.120   |   0.61: 109 ->  one hundred ninety <EOS> (one hundred nine)
 30000  40% (  3m 26s)   0.772   |   0.16: 15th -> (forcing) fifteenth <EOS> (fifteenth)
 40000  60% (  5m 11s)   0.573   |   0.13: 275 ->  two hundred seventy five <EOS> (two hundred seventy five)
 50000  80% (  6m 56s)   0.531   |   0.08: 2004 ->  two thousand four <EOS> (two thousand four)
Accuracy: 65.30% (    6530/   10000)
Saving: data/models/numbers_gen_3/50000 (encoder/decoder)
 60000 100% (  9m 16s)   0.446   |   0.02: 94.5% -> (forcing) ninety four point five percent <EOS> (ninety four point five percent)


In [37]:
test_model_accuracy(10, print_wrongs=True)

12am                 -> twelve millimeters 
                     != twelve a m
1018.1               -> one thousand one hundred eighteen 
                     != one thousand eighteen point one
¥4,000               -> four thousand pounds 
                     != four thousand yen
300                  -> three o o 
                     != three hundred
10 December 1969     -> the tenth of september nineteen ninety six nine 
                     != the tenth of december nineteen sixty nine
Accuracy: 50.00% (       5/      10)


In [38]:
train_iterations(n_iters=50000, teacher_forcing_ratio=0.4)

 70000  20% (  1m 44s)   0.475   |   0.08: 1891 ->  eighteen ninety one <EOS> (eighteen ninety one)
 80000  40% (  3m 29s)   0.406   |   0.02: 500 ->  five hundred <EOS> (five hundred)
 90000  60% (  5m 14s)   0.372   |   0.02: 18.50 -> (forcing) eighteen point five o <EOS> (eighteen point five o)
100000  80% (  6m 58s)   0.350   |   0.01: 40 million -> (forcing) forty million <EOS> (forty million)
Accuracy: 75.85% (    7585/   10000)
Saving: data/models/numbers_gen_3/100000 (encoder/decoder)
110000 100% (  9m 20s)   0.334   |   0.00: 2 ->  two <EOS> (two)


In [39]:
test_model_accuracy(10, print_wrongs=True)

1991/94              -> one thousand nine hundred ninety nine forty 
                     != one thousand nine hundred ninety one ninety fourths
0-7018-1330          -> o sil eight seven one one sil one one three three 
                     != o sil seven o one eight sil one three three o
Accuracy: 80.00% (       8/      10)


In [40]:
train_iterations(n_iters=50000, teacher_forcing_ratio=0.3)

120000  20% (  1m 44s)   0.342   |   2.25: 1996 ->  one ninety nine six (nineteen ninety six)
130000  40% (  3m 28s)   0.336   |   0.01: 15th ->  fifteenth <EOS> (fifteenth)
140000  60% (  5m 12s)   0.307   |   0.20: 2011 ->  twenty eleven <EOS> (twenty eleven)
150000  80% (  6m 57s)   0.267   |   0.58: 2013 ->  twenty o <EOS> (two o one three)
Accuracy: 79.22% (    7922/   10000)
Saving: data/models/numbers_gen_3/150000 (encoder/decoder)
160000 100% (  9m 19s)   0.267   |   0.00: 3 ->  three <EOS> (three)


In [41]:
test_model_accuracy(20, print_wrongs=True)

2009                 -> two thousand nine 
                     != two o o nine
2001                 -> two thousand one 
                     != two o o one
508                  -> five hundred eight 
                     != five o eight
Accuracy: 85.00% (      17/      20)


In [42]:
train_iterations(n_iters=50000, teacher_forcing_ratio=0.2)

170000  20% (  1m 43s)   0.296   |   0.01: 5.7 ->  five point seven <EOS> (five point seven)
180000  40% (  3m 27s)   0.290   |   0.01: 15 ->  fifteen <EOS> (fifteen)
190000  60% (  5m 12s)   0.245   |   0.01: 2.1 m ->  two point one meters <EOS> (two point one meters)
200000  80% (  6m 56s)   0.249   |   0.02: 100 ->  one hundred <EOS> (one hundred)
Accuracy: 81.87% (    8187/   10000)
Saving: data/models/numbers_gen_3/200000 (encoder/decoder)
210000 100% (  9m 17s)   0.251   |   0.10: 1978 ->  nineteen seventy eight <EOS> (nineteen seventy eight)


In [43]:
test_model_accuracy(20, print_wrongs=True)

0-86358-084          -> o sil eight six eight eight eight sil eight o four 
                     != o sil eight six three five eight sil o eight four
1997                 -> nineteen ninety seven 
                     != one nine nine seven
Accuracy: 90.00% (      18/      20)


In [44]:
train_iterations(n_iters=50000, teacher_forcing_ratio=0.2, lr=0.0001)

220000  20% (  1m 44s)   0.244   |   0.00: $25,000 ->  twenty five thousand dollars <EOS> (twenty five thousand dollars)
230000  40% (  3m 28s)   0.231   |   0.00: 3 -> (forcing) three <EOS> (three)
240000  60% (  5m 13s)   0.175   |   0.00: 4th ->  fourth <EOS> (fourth)
250000  80% (  6m 58s)   0.244   |   0.00: 3rd ->  third <EOS> (third)
Accuracy: 85.49% (    8549/   10000)
Saving: data/models/numbers_gen_3/250000 (encoder/decoder)
260000 100% (  9m 18s)   0.243   |   0.09: 2007 ->  two thousand seven <EOS> (two thousand seven)


In [45]:
test_model_accuracy(20, print_wrongs=True)

14.00 pm             -> fourteen p m 
                     != two p m
1895                 -> eighteen ninety five 
                     != one eight nine five
Accuracy: 90.00% (      18/      20)


In [46]:
train_iterations(n_iters=50000, teacher_forcing_ratio=0.4, lr=0.0001)

270000  20% (  1m 44s)   0.210   |   0.23: I -> (forcing) the first <EOS> (the first)
280000  40% (  3m 29s)   0.171   |   0.00: 0.83% ->  zero point eight three percent <EOS> (zero point eight three percent)
290000  60% (  5m 14s)   0.190   |   0.00: 1.0 -> (forcing) one point zero <EOS> (one point zero)
300000  80% (   7m 0s)   0.176   |   0.04: 818th ->  eight hundred eighteenth <EOS> (eight hundred eighteenth)
Accuracy: 86.56% (    8656/   10000)
Saving: data/models/numbers_gen_3/300000 (encoder/decoder)
310000 100% (  9m 22s)   0.156   |   0.02: $458,000 ->  four hundred fifty eight thousand dollars <EOS> (four hundred fifty eight thousand dollars)


In [47]:
test_model_accuracy(20, print_wrongs=True)

088 127 085          -> o eight eight one sil two eight seven eight five 
                     != o eight eight sil one two seven sil o eight five
Accuracy: 95.00% (      19/      20)


In [48]:
train_iterations(n_iters=50000, teacher_forcing_ratio=0.2, lr=0.001)

320000  20% (  1m 45s)   0.245   |   0.04: 54/1 ->  fifty four over one <EOS> (fifty four over one)
330000  40% (  3m 30s)   0.293   |   0.05: 1942 -> (forcing) nineteen forty two <EOS> (nineteen forty two)
340000  60% (  5m 15s)   0.268   |   0.00: .67 ->  point six seven <EOS> (point six seven)
350000  80% (   7m 0s)   0.260   |   0.72: 3000 ->  three thousand <EOS> (three o o o)
Accuracy: 82.48% (    8248/   10000)
Saving: data/models/numbers_gen_3/350000 (encoder/decoder)
360000 100% (  9m 21s)   0.286   |   0.03: 1995 ->  nineteen ninety five <EOS> (nineteen ninety five)


In [49]:
test_model_accuracy(20, print_wrongs=True)

2010                 -> twenty ten 
                     != two thousand ten
Accuracy: 95.00% (      19/      20)


In [50]:
train_iterations(n_iters=100000, teacher_forcing_ratio=0.1, lr=0.0001)

370000  10% (  1m 44s)   0.223   |   0.01: 03 ->  o three <EOS> (o three)
380000  20% (  3m 29s)   0.248   |   0.00: 12% ->  twelve percent <EOS> (twelve percent)
390000  30% (  5m 13s)   0.197   |   0.03: 04 ->  o four <EOS> (o four)
400000  40% (  6m 56s)   0.212   |   0.16: (2009)22 ->  two o o nine sil two two <EOS> (two o o nine sil two two)
Accuracy: 86.92% (    8692/   10000)
Saving: data/models/numbers_gen_3/400000 (encoder/decoder)
410000  50% (  9m 16s)   0.190   |   0.13: 598 ->  five hundred ninety eight <EOS> (five hundred ninety eight)
420000  60% (  11m 0s)   0.186   |   0.02: $19,105 ->  nineteen thousand one hundred five dollars <EOS> (nineteen thousand one hundred five dollars)
430000  70% ( 12m 44s)   0.206   |   0.02: 35 ->  thirty five <EOS> (thirty five)
440000  80% ( 14m 28s)   0.194   |   0.00: £100,000 ->  one hundred thousand pounds <EOS> (one hundred thousand pounds)
450000  90% ( 16m 12s)   0.184   |   0.00: 3 ->  three <EOS> (three)
Accuracy: 87.62% (    87

In [51]:
test_model_accuracy(20, print_wrongs=True)

978-0-8493-4258-5    -> nine seven eight sil o sil eight four nine four sil three three two five sil five 
                     != nine seven eight sil o sil eight four nine three sil four two five eight sil five
2011-04-13           -> the first of april twenty thirteen 
                     != the thirteenth of april twenty eleven
16                   -> sixteen 
                     != one six
Accuracy: 85.00% (      17/      20)


In [52]:
train_iterations(n_iters=200000, teacher_forcing_ratio=0.1, lr=0.0001)

470000   5% (  1m 43s)   0.189   |   0.00: 125th ->  one hundred twenty fifth <EOS> (one hundred twenty fifth)
480000  10% (  3m 28s)   0.185   |   0.00: $10 billion ->  ten billion dollars <EOS> (ten billion dollars)
490000  15% (  5m 12s)   0.174   |   0.02: 5 km2 ->  five square kilometers <EOS> (five square kilometers)
500000  20% (  6m 56s)   0.152   |   0.08: 2009 ->  two thousand nine <EOS> (two thousand nine)
Accuracy: 87.50% (    8750/   10000)
Saving: data/models/numbers_gen_3/500000 (encoder/decoder)
510000  25% (  9m 17s)   0.170   |   0.04: 8.022 ->  eight point o two two <EOS> (eight point o two two)
520000  30% (  11m 1s)   0.181   |   0.00: 3- ->  three <EOS> (three)
530000  35% ( 12m 46s)   0.194   |   0.04: 1877 ->  eighteen seventy seven <EOS> (eighteen seventy seven)
540000  40% ( 14m 31s)   0.180   |   0.00: 2.5 ->  two point five <EOS> (two point five)
550000  45% ( 16m 15s)   0.173   |   0.00: 1st ->  first <EOS> (first)
Accuracy: 87.55% (    8755/   10000)
Savin

In [53]:
test_model_accuracy(20, print_wrongs=True)

44-9-SIM             -> four four sil nine sil p a 
                     != four four sil nine sil sim
978-2723467049       -> nine seven eight sil two seven seven six six four four o four nine 
                     != nine seven eight sil two seven two three four six seven o four nine
Accuracy: 90.00% (      18/      20)


In [54]:
train_iterations(n_iters=200000, teacher_forcing_ratio=0.1, lr=0.0001)

670000   5% (  1m 45s)   0.154   |   0.01: C11 ->  c eleven <EOS> (c eleven)
680000  10% (  3m 28s)   0.204   |   0.00: 1 ->  one <EOS> (one)
690000  15% (  5m 13s)   0.150   |   0.00: 001 ->  o o one <EOS> (o o one)
700000  20% (  6m 58s)   0.139   |   0.00: .5 ->  point five <EOS> (point five)
Accuracy: 88.43% (    8843/   10000)
Saving: data/models/numbers_gen_3/700000 (encoder/decoder)
710000  25% (  9m 18s)   0.140   |   0.03: $11,667 ->  eleven thousand six hundred sixty seven dollars <EOS> (eleven thousand six hundred sixty seven dollars)
720000  30% (  11m 3s)   0.156   |   0.00: 2 ->  two <EOS> (two)
730000  35% ( 12m 47s)   0.191   |   0.58: 25 July 1712 ->  the twenty fifth of july twenty twelve <EOS> (the twenty fifth of july seventeen twelve)
740000  40% ( 14m 31s)   0.168   |   0.00: $3M ->  three million dollars <EOS> (three million dollars)
750000  45% ( 16m 15s)   0.184   |   0.15: 0.45 V ->  zero point four five volts <EOS> (zero point four five volts)
Accuracy: 88.84

In [55]:
test_model_accuracy(20, print_wrongs=True)

604-606 DOI          -> six o four sil six six six sil sixty 
                     != six o four sil six o six sil doi
0500 700 700         -> o five o sil seven seven o sil o 
                     != o five hundred sil seven hundred sil seven hundred
0-87195-109-6        -> o sil eight seven one five nine sil one nine nine sil six 
                     != o sil eight seven one nine five sil one o nine sil six
Accuracy: 85.00% (      17/      20)


In [56]:
train_iterations(n_iters=200000, teacher_forcing_ratio=0.1, lr=0.0001)

870000   5% (  1m 44s)   0.136   |   0.00: 2.35 ->  two point three five <EOS> (two point three five)
880000  10% (  3m 30s)   0.167   |   0.27: 0-521-78220-1 ->  o sil five two one sil two seven two two o sil one <EOS> (o sil five two one sil seven eight two two o sil one)
890000  15% (  5m 15s)   0.209   |   0.02: 22 -> (forcing) twenty two <EOS> (twenty two)
900000  20% (   7m 0s)   0.164   |   0.12: 10 ->  ten <EOS> (ten)
Accuracy: 88.93% (    8893/   10000)
Saving: data/models/numbers_gen_3/900000 (encoder/decoder)
910000  25% (  9m 21s)   0.141   |   0.00: 5.7% ->  five point seven percent <EOS> (five point seven percent)
920000  30% (  11m 7s)   0.149   |   0.01: 5-1-2 ->  five sil one sil two <EOS> (five sil one sil two)
930000  35% ( 12m 53s)   0.140   |   0.01: 80 hp ->  eighty horsepower <EOS> (eighty horsepower)
940000  40% ( 14m 37s)   0.147   |   0.00: 2.14 ->  two point one four <EOS> (two point one four)
950000  45% ( 16m 22s)   0.153   |   0.00: 2 ->  two <EOS> (two)
A

In [61]:
test_model_accuracy(20, print_wrongs=True)

Accuracy: 100.00% (      20/      20)


In [62]:
train_iterations(n_iters=200000, teacher_forcing_ratio=0, lr=0.0001)

1070000   5% (  1m 51s)   0.174   |   0.00: 7 ->  seven <EOS> (seven)
1080000  10% (  3m 38s)   0.136   |   0.00: 3 million ->  three million <EOS> (three million)
1090000  15% (  5m 28s)   0.159   |   0.95: 49 ->  forty nine <EOS> (four nine)
1100000  20% (  7m 18s)   0.173   |   0.00: 6th ->  sixth <EOS> (sixth)
Accuracy: 89.33% (    8933/   10000)
Saving: data/models/numbers_gen_3/1100000 (encoder/decoder)
1110000  25% (  9m 45s)   0.123   |   0.09: 2009 ->  two thousand nine <EOS> (two thousand nine)
1120000  30% ( 11m 36s)   0.131   |   0.00: 2 ->  two <EOS> (two)
1130000  35% ( 13m 24s)   0.146   |   0.04: 22 ->  twenty two <EOS> (twenty two)
1140000  40% ( 15m 13s)   0.138   |   0.00: 64.2 ->  sixty four point two <EOS> (sixty four point two)
1150000  45% (  17m 2s)   0.168   |   0.00: 10th ->  tenth <EOS> (tenth)
Accuracy: 89.73% (    8973/   10000)
Saving: data/models/numbers_gen_3/1150000 (encoder/decoder)
1160000  50% ( 19m 29s)   0.139   |   0.15: 1991 ->  nineteen ninety o

In [63]:
test_model_accuracy(20, print_wrongs=True)

1966 MATS            -> one thousand nine hundred sixty six million fifty 
                     != one thousand nine hundred sixty six million austrian schillings
978-3-540-77758-8    -> nine seven eight sil three sil five seven o sil seven seven seven five eight sil eight 
                     != nine seven eight sil three sil five four o sil seven seven seven five eight sil eight
2012                 -> twenty twelve 
                     != two o one two
Accuracy: 85.00% (      17/      20)


In [64]:
train_iterations(n_iters=400000, teacher_forcing_ratio=0, lr=0.0001)

1270000   2% (  1m 51s)   0.126   |   0.87: 0156029081 ->  o one six five o two o o eight eight one (o one five six o two nine o eight one)
1280000   5% (  3m 43s)   0.131   |   0.05: 1999 ->  nineteen ninety nine <EOS> (nineteen ninety nine)
1290000   8% (  5m 35s)   0.146   |   0.32: 978-0-570-01306-8 ->  nine seven eight sil o sil five seven o sil one one o three six sil eight <EOS> (nine seven eight sil o sil five seven o sil o one three o six sil eight)
1300000  10% (  7m 28s)   0.136   |   0.00: 2.3 million ->  two point three million <EOS> (two point three million)
Accuracy: 89.60% (    8960/   10000)
Saving: data/models/numbers_gen_3/1300000 (encoder/decoder)
1310000  12% (  9m 53s)   0.139   |   0.08: 607.5 KB ->  six hundred seven point five kilobytes <EOS> (six hundred seven point five kilobytes)
1320000  15% ( 11m 42s)   0.132   |   0.01: 0.54% ->  zero point five four percent <EOS> (zero point five four percent)
1330000  18% ( 13m 32s)   0.171   |   0.00: 8.78 ->  eight po

# 

In [ ]:
1+1

In [66]:
test_model_accuracy(200, print_wrongs=True)

X                    -> the tenth 
                     != tenth
0-415-30779-1        -> o sil four one five sil three seven seven seven nine sil one 
                     != o sil four one five sil three o seven seven nine sil one
74321 41064 2        -> seven four two three sil one one one four o o sil two 
                     != seven four three two one sil four one o six four sil two
107.5 KZL            -> one hundred seven point five thousand fifteen 
                     != one hundred seven point five thousand zlotys
0-7190-6122-9        -> o sil seven one o nine sil one six two two sil nine 
                     != o sil seven one nine o sil six one two two sil nine
214/209PN            -> two hundred fourteen two hundred ten hundred forty 
                     != two hundred fourteen two hundred ninths of a peta newton
1969                 -> nineteen sixty nine 
                     != one nine six nine
1994                 -> nineteen ninety nine four 
                    

In [67]:
train_iterations(n_iters=600000, teacher_forcing_ratio=0, lr=0.0001)

1670000   2% (  1m 48s)   0.158   |   0.00: 3:50 ->  three fifty <EOS> (three fifty)
1680000   3% (  3m 39s)   0.146   |   0.01: March 20, 2016 ->  march twentieth twenty sixteen <EOS> (march twentieth twenty sixteen)
1690000   5% (  5m 25s)   0.106   |   0.00: 0.26% ->  zero point two six percent <EOS> (zero point two six percent)
1700000   7% (  7m 14s)   0.134   |   0.00: 3rd ->  third <EOS> (third)
Accuracy: 89.56% (    8956/   10000)
Saving: data/models/numbers_gen_3/1700000 (encoder/decoder)
1710000   8% (  9m 36s)   0.134   |   0.00: 135 mph ->  one hundred thirty five miles per hour <EOS> (one hundred thirty five miles per hour)
1720000  10% ( 11m 20s)   0.147   |   0.00: 1,377.9 ->  one thousand three hundred seventy seven point nine <EOS> (one thousand three hundred seventy seven point nine)
1730000  12% (  13m 5s)   0.121   |   0.27: 0-87972-474-9 ->  o sil eight seven nine seven two sil seven four four sil nine <EOS> (o sil eight seven nine seven two sil four seven four sil

KeyboardInterrupt: 

In [ ]:
test_model_accuracy(20, print_wrongs=True)

In [ ]:
train_iterations(n_iters=600000, teacher_forcing_ratio=0, lr=0.0001)

In [ ]:
test_model_accuracy(20, print_wrongs=True)

In [ ]:
train_iterations(n_iters=600000, teacher_forcing_ratio=0, lr=0.0001)

In [ ]:
test_model_accuracy(1000, print_wrongs=True)

In [ ]:
all_data[all_data['sentence_id'].isin(all_data[all_data['after'] == "two o one o"].sample(1)['sentence_id'])]

In [ ]:
all_data[all_data['sentence_id'].isin(all_data[all_data['after'] == "twenty ten"].sample(1)['sentence_id'])]